In [1]:
import json
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import pearsonr, spearmanr

In [2]:
corona_df=pd.read_csv('../data/raw/corona/de_corona.csv',sep='\t')

with open('../data/raw/metadata/de_metadata.json','r', encoding="utf8") as f:
    country_metadata=json.load(f)

region_map = {country_metadata["country_metadata"][i]["covid_region_code"]: 
    country_metadata["country_metadata"][i]["iso3166-2_code"] for i in range(len(country_metadata["country_metadata"]))}
corona_df["iso3166-2"] = corona_df["region_code"].map(region_map)


weather_df = pd.read_csv("../data/raw/weather/weather.csv")

weather_df["TemperatureAboveGround"] = weather_df["TemperatureAboveGround"] - 273.15
weather_df = weather_df[weather_df["iso3166-2"].str.startswith("DE")]

df = corona_df.merge(weather_df)

df

,date,region_code,confirmed_addition,deceased_addition,iso3166-2,RelativeHumiditySurface,SolarRadiation,Surfacepressure,TemperatureAboveGround,Totalprecipitation,UVIndex,WindSpeed
0,2020-02-16,Mecklenburg-Vorpommern,1,0,DE-MV,69.818665,4.562424e+04,2.392858e+06,11.144049,0.004641,0.211216,7.888859
1,2020-02-17,Brandenburg,1,0,DE-BB,64.189273,9.007504e+05,2.409194e+06,9.162595,0.001005,2.631685,6.319781
2,2020-02-18,Brandenburg,1,0,DE-BB,71.904630,4.042940e+06,2.421421e+06,6.529468,0.002388,4.037911,6.150824
3,2020-02-20,Nordrhein-Westfalen,1,0,DE-NW,81.738535,1.064802e+06,2.383996e+06,6.513867,0.006802,3.524003,6.477330
4,2020-02-24,Baden-Württemberg,1,0,DE-BW,80.314208,6.604918e+05,2.308155e+06,8.744530,0.005284,5.613589,4.663207
...,...,...,...,...,...,...,...,...,...,...,...,...
5575,2021-02-21,Rheinland-Pfalz,120,0,DE-RP,65.642820,8.457394e+06,2.348996e+06,8.973657,0.000000,7.879810,3.009041
5576,2021-02-21,Sachsen,68,0,DE-SN,74.638170,6.687276e+06,2.363996e+06,6.127820,0.000008,5.409593,4.648033
5577,2021-02-21,Sachsen-Anhalt,84,0,DE-ST,69.587170,7.276057e+06,2.408002e+06,9.268775,0.000000,5.273630,2.972845
5578,2021-02-21,Schleswig-Holstein,113,0,DE-SH,78.077610,5.698494e+06,2.431487e+06,8.793486,0.000000,4.474084,2.532838
